<a href="https://colab.research.google.com/github/caio-torres-seares/primeira-rede-YOLO/blob/main/primeira_rede_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
# Instalando api do coco
!pip install pycocotools
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.7 MB/s eta 0:00:00


In [25]:
import torch
from ultralytics import YOLO
import yaml
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import shutil

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
# Clona o repositório do darknet para o ambiente do Colab
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 15909, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 15909 (delta 2), reused 2 (delta 2), pack-reused 15903 (from 2)
Receiving objects: 100% (15909/15909), 14.46 MiB | 15.46 MiB/s, done.
Resolving deltas: 100% (10705/10705), done.


In [ ]:
# Entra na pasta do darknet
%cd darknet

# Modifica o Makefile para habilitar GPU, CUDNN e OpenCV
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

# Compila o darknet (isso pode levar alguns minutos)
!make

In [5]:
!mkdir -p data/annotations
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip -P data/
!unzip data/annotations_trainval2017.zip -d data/

--2025-09-20 00:42:37--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.12.113, 16.182.34.65, 3.5.29.214, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.12.113|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘data/annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  16.8MB/s    in 16s     

2025-09-20 00:42:54 (14.6 MB/s) - ‘data/annotations_trainval2017.zip’ saved [252907541/252907541]

Archive:  data/annotations_trainval2017.zip
  inflating: data/annotations/instances_train2017.json  
  inflating: data/annotations/instances_val2017.json  
  inflating: data/annotations/captions_train2017.json  
  inflating: data/annotations/captions_val2017.json  
  inflating: data/annotations/person_keypoints_train2017.json  
  inflating: data/annotations/person_keypoints_val2017.json  


In [14]:
from pycocotools.coco import COCO
import requests, os

# caminho para as anotações baixadas
annFile = 'data/annotations/instances_train2017.json'
coco = COCO(annFile)

categories = ['dog', 'car']
catIds = coco.getCatIds(catNms=categories)
imgIds = coco.getImgIds(catIds=catIds)

print(f"Total de imagens encontradas: {len(imgIds)}")
print(f"Categorias: {categories}")
print(f"IDs das categorias: {catIds}")

os.makedirs('data/coco_subset/images/train', exist_ok=True)
os.makedirs('data/coco_subset/images/val', exist_ok=True)
os.makedirs('data/coco_subset/labels/train', exist_ok=True)
os.makedirs('data/coco_subset/labels/val', exist_ok=True)

loading annotations into memory...
Done (t=13.11s)
creating index...
index created!
Total de imagens encontradas: 481
Categorias: ['dog', 'car']
IDs das categorias: [3, 18]


In [10]:
def download_images(img_ids, split):
    """
    Baixa imagens para um split específico (train/val)
    """
    for i, imgId in enumerate(img_ids):
        try:
            img = coco.loadImgs(imgId)[0]
            img_data = requests.get(img['coco_url']).content

            # Salvar na pasta correta (train ou val)
            img_path = f"data/coco_subset/images/{split}/{img['file_name']}"
            with open(img_path, 'wb') as handler:
                handler.write(img_data)

            if (i + 1) % 10 == 0:
                print(f"  {split}: {i + 1} imagens baixadas")

        except Exception as e:
            print(f"Erro ao baixar imagem {imgId}: {e}")


In [11]:
def download_and_split_dataset(num_images=200):  # Aumentei para 200 imagens
    """
    Baixa imagens e divide em treino/validação (80/20)
    """

    # Dividir IDs em treino e validação
    train_ids, val_ids = train_test_split(
        imgIds[:num_images],
        test_size=0.2,
        random_state=42
    )

    print(f"Imagens de treino: {len(train_ids)}")
    print(f"Imagens de validação: {len(val_ids)}")

    # Baixar imagens de treino
    print("Baixando imagens de treino...")
    download_images(train_ids, 'train')

    # Baixar imagens de validação
    print("Baixando imagens de validação...")
    download_images(val_ids, 'val')

    return train_ids, val_ids

In [16]:
train_ids, val_ids = download_and_split_dataset(200)

Imagens de treino: 160
Imagens de validação: 40
Baixando imagens de treino...
  train: 10 imagens baixadas
  train: 20 imagens baixadas
  train: 30 imagens baixadas
  train: 40 imagens baixadas
  train: 50 imagens baixadas
  train: 60 imagens baixadas
  train: 70 imagens baixadas
  train: 80 imagens baixadas
  train: 90 imagens baixadas
  train: 100 imagens baixadas
  train: 110 imagens baixadas
  train: 120 imagens baixadas
  train: 130 imagens baixadas
  train: 140 imagens baixadas
  train: 150 imagens baixadas
  train: 160 imagens baixadas
Baixando imagens de validação...
  val: 10 imagens baixadas
  val: 20 imagens baixadas
  val: 30 imagens baixadas
  val: 40 imagens baixadas


In [17]:
def create_yolo_labels(img_ids, split):
    """
    Cria labels YOLO para um split específico
    """
    print(f"Criando labels para {split}...")

    for i, imgId in enumerate(img_ids):
        try:
            img_info = coco.loadImgs(imgId)[0]
            img_name = img_info['file_name']
            img_width = img_info['width']
            img_height = img_info['height']

            annIds = coco.getAnnIds(imgIds=imgId, catIds=catIds, iscrowd=None)
            anns = coco.loadAnns(annIds)

            lines = []
            for ann in anns:
                x, y, w, h = ann['bbox']  # bbox no formato COCO

                # Converter para formato YOLO (normalizado)
                x_center = (x + w/2) / img_width
                y_center = (y + h/2) / img_height
                w_norm = w / img_width
                h_norm = h / img_height

                # Mapear categoria para classe (0=dog, 1=car)
                cat_name = coco.loadCats(ann['category_id'])[0]['name']
                class_id = categories.index(cat_name)

                lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}")

            # Salvar arquivo .txt
            label_path = f"data/coco_subset/labels/{split}/{img_name.replace('.jpg', '.txt')}"
            with open(label_path, 'w') as f:
                f.write("\n".join(lines))

        except Exception as e:
            print(f"Erro ao processar label {imgId}: {e}")

    print(f"Labels {split} criados com sucesso!")

In [18]:
# Criar labels para ambos os splits
create_yolo_labels(train_ids, 'train')
create_yolo_labels(val_ids, 'val')

Criando labels para train...
Labels train criados com sucesso!
Criando labels para val...
Labels val criados com sucesso!


In [29]:
def create_dataset_yaml():
    """
    Cria arquivo de configuração do dataset
    """
    dataset_config = {
        'path': '/content/data/coco_subset',  # caminho raiz do dataset
        'train': 'images/train',              # caminho relativo para treino
        'val': 'images/val',                  # caminho relativo para validação
        'nc': 2,                              # número de classes
        'names': ['dog', 'car']               # nomes das classes
    }

    yaml_path = 'data/coco_subset/dataset.yaml'
    with open(yaml_path, 'w') as f:
        yaml.dump(dataset_config, f, default_flow_style=False)

    print(f"Arquivo dataset.yaml criado: {yaml_path}")
    return yaml_path

In [30]:
yaml_path = create_dataset_yaml()

Arquivo dataset.yaml criado: data/coco_subset/dataset.yaml


In [31]:
def train_yolo_model():
    """
    Treina modelo YOLO com seus dados
    """
    # Carregar modelo pré-treinado
    model = YOLO('yolov8n.pt')  # YOLOv8 nano (rápido para teste)

    print("Iniciando treinamento YOLO...")
    print("Configurações:")
    print(f"- Modelo: YOLOv8n")
    print(f"- Classes: {categories}")
    print(f"- Épocas: 100")
    print(f"- Batch size: 16")
    print(f"- Imagem: 640x640")

    # Treinar modelo
    results = model.train(
        data=yaml_path,           # seu dataset.yaml
        epochs=100,               # número de épocas
        imgsz=640,                # tamanho da imagem
        batch=16,                 # batch size
        device=0 if torch.cuda.is_available() else 'cpu',
        project='yolo_results',   # pasta dos resultados
        name='dog_car_detector',  # nome do experimento
        save=True,                # salvar checkpoints
        plots=True,               # gerar gráficos
        patience=10,              # early stopping
        save_period=10,           # salvar a cada 10 épocas
        verbose=True              # logs detalhados
    )

    print("Treinamento concluído!")
    return model, results

In [32]:
# Executar treinamento
trained_model, training_results = train_yolo_model()

Iniciando treinamento YOLO...
Configurações:
- Modelo: YOLOv8n
- Classes: ['dog', 'car']
- Épocas: 100
- Batch size: 16
- Imagem: 640x640
Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/coco_subset/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=d

In [36]:
def validate_model(model):
    """
    Valida o modelo treinado - VERSÃO CORRIGIDA
    """
    print("Validando modelo...")

    # Executar validação
    val_results = model.val()

    print("Métricas de Validação:")
    print(f"mAP50: {val_results.box.map50:.4f}")
    print(f"mAP50-95: {val_results.box.map:.4f}")

    try:
        if hasattr(val_results.box, 'mp') and hasattr(val_results.box, 'mr'):
            mp = val_results.box.mp
            mr = val_results.box.mr

            print(f"\nTipo de mp: {type(mp)}")
            print(f"Valor de mp: {mp}")
            print(f"Tipo de mr: {type(mr)}")
            print(f"Valor de mr: {mr}")

            if hasattr(mp, '__len__') and len(mp) > 1:
                print("\nMétricas por classe:")
                for i, class_name in enumerate(categories):
                    if i < len(mp):
                        precision = mp[i]
                        recall = mr[i]
                        print(f"{class_name}: Precision={precision:.4f}, Recall={recall:.4f}")
            else:
                print(f"\nPrecision geral: {mp:.4f}")
                print(f"Recall geral: {mr:.4f}")

        if hasattr(val_results.box, 'maps'):
            maps = val_results.box.maps
            if hasattr(maps, '__len__') and len(maps) > 1:
                print("\nmAP50 por classe:")
                for i, class_name in enumerate(categories):
                    if i < len(maps):
                        print(f"{class_name}: {maps[i]:.4f}")

        print(f"\nInformações adicionais:")
        if hasattr(val_results.box, 'nc'):
            print(f"Número de classes: {val_results.box.nc}")
        if hasattr(val_results.box, 'nt'):
            print(f"Total de targets: {val_results.box.nt}")

    except Exception as e:
        print(f"Erro ao acessar métricas detalhadas: {e}")
        print("Mostrando apenas métricas principais.")

    return val_results


In [37]:
# Validar modelo
validation_results = validate_model(trained_model)

Validando modelo...
Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1539.6±275.6 MB/s, size: 161.5 KB)
val: Scanning /content/darknet/data/coco_subset/labels/val.cache... 40 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 40/40 68.5Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 2.0it/s 1.5s
                   all         40        167      0.817      0.144      0.363      0.237
                   dog         40         53          1      0.262      0.552       0.38
                   car         40        114      0.633     0.0263      0.174      0.095
Speed: 6.3ms preprocess, 9.8ms inference, 0.0ms loss, 6.1ms postprocess per image
Results saved to /content/darknet/yolo_results/dog_car_detector24
Métricas de Validação:
mAP50: 0.3634
mAP50-95: 0.2373

Tipo de mp: <class 'numpy.float64'>
Valor de mp: 0.8166612155435061
Tipo 